In [ ]:
# Feature Importance: Justification for Key Features

https://medium.com/data-science-journal/how-to-measure-feature-importance-in-a-binary-classification-model-d284b8c9a301

In [1]:
# Loading 

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn import metrics
from itertools import combinations
import itertools
import json

# 1 -- Load Data

# Windowed Data: 1 Second Overlapping Windows, Feature Median + Variance in Window 
p5_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p5_master_window.csv'
p7_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p7_master_window.csv'
p9_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p9_master_window.csv'
p11_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p11_master_window.csv'
p12_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p12_master_window.csv'
p17_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p17_master_window.csv'
p18_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p18_master_window.csv'

data5 = pd.read_csv(p5_file)
data7 = pd.read_csv(p7_file)
data9 = pd.read_csv(p9_file)
data11 = pd.read_csv(p11_file)
data12 = pd.read_csv(p12_file)
data17 = pd.read_csv(p17_file)
data18 = pd.read_csv(p18_file)

data = [data5, data7, data9, data11, data12, data17, data18]
part = [5, 7, 9, 11, 12, 17, 18]

remove = [
'engagement_change',
'ros_GAME_STATE',
'ros_PARTICIPANT_STATE',
'ros_ROBOT_STATE',
'ros_activity',
'ros_diff_1_change',
'ros_diff_2_change',
'ros_diff_3_change',
'ros_diff_4_change',
'ros_diff_5_change',
'ros_difficulty',
'ros_skill_EM_change',
'ros_skill_NC_change',
'ros_skill_OS_change',
'ros_ts_attempt_var',
'ros_games_session_change',
'ros_in_game_change',
'ros_mistakes_session_change',
'ros_mistakes_game_change',
'ros_skill',
'ros_ts_game_start_var',
'ros_ts_robot_talked_var',
'ros_game_correct',
'ros_game_incorrect',
'ros_game_start',
'ros_mistake_made',
]

for i, d in enumerate(data):
    data[i] = d.drop(columns=remove)

# Remove rows where engagment NaN
for i,d in enumerate(data):
    data[i] = d[np.isfinite(d['engagement'])]
    
# Remove rows where engagment is -1
for i,d in enumerate(data):
    data[i] = d[d['engagement']>=0]
    
all_data = pd.concat(data, ignore_index=True, sort=True)

In [2]:
basic_features = ['participant', 'session_num', 'engagement']
binary_features = []
numerical_features = []
for c in all_data.columns:
    if 'AU' in c or '_change' in c or '_skill' in c or '_diff' in c or c in ['of_success', 'ros_in_game']:
        binary_features.append(c)
    elif c not in basic_features:
        numerical_features.append(c)

In [3]:
nan_max_cols = ['of_gaze_0_x',
'of_gaze_0_y',
'of_gaze_0_z',
'of_gaze_1_x',
'of_gaze_1_y',
'of_gaze_1_z',
'of_gaze_angle_x',
'of_gaze_angle_y',
'of_gaze_distance',
'of_gaze_distance_x',
'of_gaze_distance_y',
'of_pose_Rxv',
'of_pose_Ry',
'of_pose_Rz',
'of_pose_Tx',
'of_pose_Ty',
'of_pose_Tz',
'of_pose_distance']

In [4]:
for c in all_data.columns:
    if c in binary_features or c in numerical_features:
        mean = np.nanmean(all_data[c])
        std = np.nanstd(all_data[c])

        if std == 0:
            all_data[c] = (all_data[c]-mean)
        else:
            all_data[c] = (all_data[c]-mean)/(std)

        if c not in nan_max_cols:
            min_val = np.nanmin(all_data[c])

            all_data[c] = all_data[c].fillna(min_val)
        else:
            max_val = np.nanmax(all_data[c])

            all_data[c] = all_data[c].fillna(max_val)

In [5]:
from scipy import stats
import operator

In [6]:
numerical_results = {}
for f in numerical_features:
    numerical_results[f] = abs(stats.pearsonr(all_data[f], all_data['engagement'])[0])
numerical_results = sorted(numerical_results.items(), key=operator.itemgetter(1))

In [12]:
for i in numerical_results:
    print(i[0], i[1])

of_ts_success_var 0.0014502406304759195
a_intensity_var 0.0017782615758854593
a_mfcc_0_var 0.004986697348104668
of_gaze_distance_x_var 0.005505187489160796
of_gaze_distance_var 0.005535664990880406
of_gaze_distance_y_var 0.006642256498713
a_pitch_strength 0.008704788901455263
a_pitch_frequency 0.008925163261466997
of_pose_Rx_var 0.011607145762594872
of_pose_Ty_var 0.011846459436433759
of_pose_Tx_var 0.012725962990906545
ros_aptitude_var 0.013051646787529955
of_pose_Rz_var 0.013359094117668403
a_pitch_strength_var 0.014423763149933738
of_pose_Tz_var 0.016288489630288974
of_pose_distance_var 0.016593175555371313
a_harmonicity_var 0.02158991723482078
of_pose_Ry_var 0.02241784996667637
a_mfcc_1 0.03117951430559799
of_gaze_angle_y_var 0.03535912183520486
a_harmonicity 0.03561208790469342
a_pitch_frequency_var 0.03818247663904372
of_gaze_1_z_var 0.043780482277260925
of_gaze_0_y_var 0.04614711930166213
a_intensity 0.046249419719211586
a_mfcc_1_var 0.04787854860987493
of_gaze_angle_x_var 0.054

In [8]:
categorical_results = {}
for f in binary_features:
    val = all_data[f].unique()[0]
    sub1 = all_data.loc[all_data[f]==val]
    sub2 = all_data.loc[all_data[f]!=val]
    
    e1 = sub1['engagement'].sum()/len(sub1)
    e2 = sub2['engagement'].sum()/len(sub2)
    
    categorical_results[f] = abs(e1-e2)
categorical_results = sorted(categorical_results.items(), key=operator.itemgetter(1))

In [9]:
for i in categorical_results:
    print(i[0], i[1])

ros_skill_EM 0.0036842821561401395
of_AU28_c 0.004041405251665187
ros_diff_1 0.012387165432729441
of_AU05_c 0.030506898280247108
ros_skill_OS 0.031005788040396
ros_skill_NC 0.03181390652030214
of_AU01_c 0.03503464104380816
ros_diff_5 0.03748800028844379
ros_diff_4 0.04158878348373185
ros_diff_3 0.06313456042934273
ros_in_game 0.07852469077357904
op_num_people_change 0.07925149167698764
of_AU17_c 0.07926061779549065
of_AU05_c_change 0.08287085439547426
of_AU26_c 0.08367141555701685
of_AU10_c 0.08965011227089525
of_success_change 0.09462759928992692
ros_diff_2 0.09541962184815644
of_AU07_c 0.09853584758850586
of_AU28_c_change 0.10142323758648786
of_AU14_c_change 0.10260054334261381
of_AU07_c_change 0.12859325362390006
of_AU15_c 0.13031566171877873
of_AU20_c 0.153730146296164
of_AU45_c 0.1564683180165658
of_AU12_c_change 0.16118302468599865
of_AU10_c_change 0.1623058772444923
of_AU23_c 0.164414300501594
of_AU01_c_change 0.16659229763964745
of_AU02_c 0.1704411414459196
of_AU02_c_change 0.1

In [10]:
key_features = [
'timestamp',
'op_num_people',
'of_pose_distance',
'of_gaze_distance',
'of_confidence',
'ros_mistakes_session',
'ros_ts_robot_talked',]

In [11]:
for f in key_features:
    for o in key_features:
        if f != o:
            print(f, o, stats.pearsonr(all_data[f], all_data[o])[0])
    print()

timestamp op_num_people -0.29731584806344935
timestamp of_pose_distance 0.25161344802350566
timestamp of_gaze_distance 0.24017227098481295
timestamp of_confidence -0.24661215096490166
timestamp ros_mistakes_session 0.768917301176784
timestamp ros_ts_robot_talked 0.29054747745519427

op_num_people timestamp -0.29731584806344935
op_num_people of_pose_distance -0.4189830930901057
op_num_people of_gaze_distance -0.3931370608537672
op_num_people of_confidence 0.3988818093419966
op_num_people ros_mistakes_session -0.2968238075471269
op_num_people ros_ts_robot_talked -0.24685979945763195

of_pose_distance timestamp 0.25161344802350566
of_pose_distance op_num_people -0.4189830930901057
of_pose_distance of_gaze_distance 0.9928299847175869
of_pose_distance of_confidence -0.967234226424457
of_pose_distance ros_mistakes_session 0.25294065032769614
of_pose_distance ros_ts_robot_talked 0.18766005035438976

of_gaze_distance timestamp 0.24017227098481295
of_gaze_distance op_num_people -0.3931370608537